# Data Exploration

In order to obtain an appropriate representation of the data, I chose to look at April through June for both 2018 and 2019. This way I would be able to gain understanding of year-over-year growth as well account for seasonality as I would be using the same time periods. In addition, I would be using some of the latest available data.

Below you will find the code I used to extract the csv files from the zip files, as well as how I cleaned up the column names and put each file into a dictionary using years as keys. I then concatenated each month file into a Q2 file for 2018 and 2019. Using each Q2 file, I created an age column as the original dataset only provided year of birth. Using each respective year, I subtracted the year of birth from it, in order to determine the age of each user. Lastly I combined the Q2 2018 and Q2 2019 files into one which I loaded into Tableau.

In Tableau, I created trip duration columns in minutes, as well as in hours, since the original column was in seconds. Using trip_duration_in_hours, I created a filter on the data to exclude all trips that exceeded 168 hours (i.e. 7 days) as a way to exclude some of the outliers in the dataset.

** Please note that I have not included any of the zip or csv files in the repository as they were too large

# Data Analysis

### Average Trip Duration: Subscriber vs Customer
When it comes to average trip duration, customers travel over two times longer than subscribers (33 min versus 13 min). Where customer travel time varies by day of the week and time of day, average subscriber travel time shows little variance. This phenomenon is also visible when it comes to looking at age groups for customers versus subscribers. The only metric where there is some variance for subscribers is when investigating gender; however, it only varies by approx. 2 minutes between males and females.

### Weekday Usage: Subscriber vs Customer
The frequency in which customers and subscribers use Citi Bike during the week varies greatly. Customers are more likely to use the service during the weekend, while subscribers use bikes during the weekdays and far less during the weekends. Upon looking at the time of day in which bikes are used, it is clear to see that during the week, subscribers prodominantly use Citi Bike around 8am and between 4pm to 6pm. This trend indicates that they likely use the service when getting to and from work and/or school. However, on the weekends, subscribers and customers follow very similar usage timelines, with a gradual increase around 10am and a decline at 4pm.


### City Official Map
Using the interactive map, notable trends include:
- The most popular station is Pershing Square North which is located near Grand Central Station. Its proximity to Grand Central Station indicates that not only New Yorkers but tourists are able to quickly access Citi Bike and ride around Manhattan.
- Stations in Manhattan are far more popular than those in Brooklyn or Queens. There are also significantly more stations there, making it easier for shorter rides.
- Many popular Citi Bike stations are also located near Central Park and as it approaches the summer months, those stations grow in popularity.

In [2]:
# Dependencies
import os
from os import listdir,stat
from os.path import isfile, join
import pandas as pd
import numpy as np
from zipfile import ZipFile 

In [5]:
# Folder path for all zip files
zip_path = "zip_data/"
# Create list of files in the zip folder
zipped_files = [f for f in listdir(zip_path) if isfile(join(zip_path, f))]

# Since only data from 2018 and 2019 will be investigated, use a for-loop to extract the csv file from 
# the zip file and input it into the csv_data/ directory given that it contains either 2018 or 2019
for file in zipped_files:
    zip_file = zip_path + file
    csv_file = file.replace('zip','csv')
    with ZipFile(zip_file,'r') as zipfile:
        if '2018' in csv_file or '2019' in csv_file:
            zipfile.extractall(path = 'csv_data/')

In [33]:
# Folder path for all csv files
csv_path = "csv_data/"
# Create list of files in the csv folder
csv_files = [f for f in listdir(csv_path) if isfile(join(csv_path, f))]

# Set the keys for the dictionary
years = ['2018','2019']

# Use a for-loop to create a 
vals = []
for year in years:
    files_in_year = [file for file in csv_files if year in file]
    vals.append(files_in_year)

# Create a file dictionary with years as keys and all the files in that year as values
file_dict = dict(zip(years, vals))
print(file_dict)

{'2018': ['201804-citibike-tripdata.csv', '201805-citibike-tripdata.csv', '201806-citibike-tripdata.csv'], '2019': ['201904-citibike-tripdata.csv', '201905-citibike-tripdata.csv', '201906-citibike-tripdata.csv']}


In [34]:
for year in years:
    combined_data = []
    for file in file_dict[year]:
        data = pd.read_csv(csv_path + file)
        if list(data.columns) != ['tripduration', 'starttime', 'stoptime', 'start station id',
                                  'start station name', 'start station latitude',
                                  'start station longitude', 'end station id', 'end station name',
                                  'end station latitude', 'end station longitude', 'bikeid', 'usertype',
                                  'birth year', 'gender']:
            data = data.rename(str.lower, axis='columns')
            data = data.rename(columns={"trip duration":"tripduration", "start time":"starttime", 
                                        "stop time":"stoptime", "bike id":"bikeid", "user type":"usertype"})
        combined_data.append(data)
    combined_data = pd.concat(combined_data)
    combined_data.to_csv('annual_data/Q2_' + year + '_data.csv')
    print('Complete file!')

Complete file!
Complete file!


In [3]:
# Create pandas dataframes for Q2 2018 and Q2 2019 data
df_q2_2018 = pd.read_csv('annual_data/Q2_2018_data.csv')
df_q2_2019 = pd.read_csv('annual_data/Q2_2019_data.csv')

In [4]:
# Create age column in each respective dataframe
df_q2_2018['age'] = 2018 - df_q2_2018['birth year']
df_q2_2019['age'] = 2019 - df_q2_2019['birth year']

# Put both dataframes into a list
q2 = [df_q2_2018, df_q2_2019]
# Concatenate both dataframe
q2_data = pd.concat(q2)

# Adjust the data type for start and stop times
q2_data['starttime'] =  pd.to_datetime(q2_data['starttime'])
q2_data['stoptime'] =  pd.to_datetime(q2_data['stoptime'])

In [9]:
q2_data.to_csv('annual_data/Q2_2018-2019_data.csv')